In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn.cluster as cluster
from sklearn.datasets import load_iris,fetch_mldata
import time
%matplotlib notebook

# Clustering

Clustering refers to unsupervised learning methods that take data points and assign them to $k$ different clusters.

There are many different kinds of clustering algorithms, for a classical review look at:

Jain, Anil K., M. Narasimha Murty, and Patrick J. Flynn. "Data clustering: a review." ACM Computing Surveys (CSUR) 31.3 (1999): 264-323.

For a more recent review investigating properties of clustering algorithms for *big data* look at:

Fahad, Adil, et al. "A survey of clustering algorithms for big data: Taxonomy and empirical analysis." IEEE Transactions on Emerging Topics in Computing 2.3 (2014): 267-279.

## Desirable characteristics of clustering algorithms

Here are some desirable characteristics of clustering algorithms (CA):

* *robustness to noise and outliers*: most real-world data contains noisy points and/or outliers that may influence the clustering. It will be desirable for a CA to be conservative in its clustering, that is, to only assign those points to cluster that actually belong there. This property - while intuitive - is actually hard to get, since that means that CAs do NOT assign cluster labels to EVERY point!

* *Occam's razor*: we can fit pretty much any distribution with enough parameters. In order to avoid over-fitting and in order for the researcher to properly control the CA, the number of its parameters should be small *and* they should be intuitively understandable so that they can be changed upon inspection of the clustering result

* *stability*: many algorithms start with random initializations. A stable CA is one that returns the same clustering result for different initializations.

* *performance with high-dimensional data*: a good CA can handle high-dimensional feature spaces

* *performance scalability*: a good CA scales from small-scale to large-scale problems.

## Taxonomy

![title](images/clusteringAlgorithmsTaxonomy.png)


## Some performance characteristics

![title](images/clusteringAlgorithmsOverview.png)


Let's test some clustering algorithms on two datasets - a 2D dataset that was designed to showcase some difficulties in clustering and our standard IRIS dataset, for which we actually know the ground truth (that is, we have 3 clusters of flowers).

In [ ]:
# load 2D data from the authors of HDBSCAN
data2D = np.load('data/clusterable_data.npy')
# load the good old IRIS dataset
iris = load_iris()

Let's look at the 2D dataset first.

In [ ]:
sb.set_context('notebook') # set to 'poster' for large-scale, high-quality output
sb.set_color_codes() # set standard colors from seaborn
plot_args = {'alpha' : 0.25, 's' : 80, 'linewidths':0} # plot data with slightly transparent look
plt.scatter(data2D.T[0],data2D.T[1],c='b',**plot_args)

## Define clustering functions

The following codes define helper functions that are designed to run a clustering using the `sklearn` interface and the to plot its result using `seaborn` functions. In addition, for the IRIS dataset, the function also counts the number of points for each of the 3 ground-truth flower classes and plots the ground-truth into the clustering (for the first 2 of the 4 dimensions in the dataset).

In [ ]:
# evaluate clustering algorithm on data
def plot_clusters2D(data, algorithm, args, kwds):
    # time the algorithm
    start_time = time.time()
    # use the algorithm based on the sklearn API with arguments and keywords supplied
    labels = algorithm(*args, **kwds).fit_predict(data)
    # time
    end_time = time.time()
    # now plot using a good color palette from seaborn initialized with the
    # number of labels found by the clustering algorithm  
    palette = sb.color_palette('deep', np.unique(labels).max() + 1)
    # assign the colors to points
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    f, ax = plt.subplots(figsize=(6,6))
    # now scatter the data with standard parameters
    ax.scatter(data.T[0], data.T[1], c=colors, **plot_args)
    # get rid of axes
    frame = f.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    # add descriptive title and print out time
    plt.suptitle('Clusters found by {} took {:.2f} s'.format(str(algorithm.__name__),end_time - start_time), fontsize=14)
    
    # evaluate clustering algorithm on data
def plot_clustersIRIS(iris_data, algorithm, args, kwds):
    # time the algorithm
    start_time = time.time()
    # use the algorithm based on the sklearn API with arguments and keywords supplied
    labels = algorithm(*args, **kwds).fit_predict(iris_data.data)
    # time
    end_time = time.time()
    # here we simply print out how many cluster labels we have for each of the three
    # ground-truth IRIS flower classes
    for label in np.unique(labels):
        c1 = len(np.where(labels[:50]==label)[0])
        c2 = len(np.where(labels[50:100]==label)[0])
        c3 = len(np.where(labels[100:]==label)[0])
        print(label,c1,c2,c3)
 
    # now plot using a good color palette from seaborn initialized with the
    # number of labels found by the clustering algorithm  
    palette = sb.color_palette('deep', np.unique(labels).max() + 1)
    # assign the colors to points
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    f, ax = plt.subplots(figsize=(6,6))
    # now scatter the data with standard parameters
    ax.scatter(iris_data.data.T[0], iris_data.data.T[1], c=colors, **plot_args)
    for idx in range(len(iris_data.data)):
        plt.text(iris_data.data.T[0][idx], iris_data.data.T[1][idx], str(iris_data.target[idx]))
    # get rid of axes
    frame = f.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    # add descriptive title and print out time
    plt.suptitle('{} found {} clusters, took {:.2f}s'.format(str(algorithm.__name__),len(np.unique(labels)),end_time - start_time), fontsize=14)


# K-means

K-means is a very simple, iterative partitioning algorithm, which takes all datapoints and assigns them into partitions in the feature space using the centroids of the partitions.

```
select k random points as centroids

repeat
    assign each point to the nearest cluster centroid
    update centroids
until (centroid location does not change)
```

* *robustness to noise and outliers*: not good

* *Occam's razor*: has one parameter, which is the number of clusters - a rather simple, but also very "knowledgable" parameter.

* *stability*: can converge to different solutions depending on the initialization - best to run multiple times

* *performance with high-dimensional data*: excellent

* *performance scalability*: usually good


In [ ]:
plot_clusters2D(data2D, cluster.KMeans, (), {'n_clusters':6})
plot_clustersIRIS(iris, cluster.KMeans, (), {'n_clusters':3})

# Mean-shift

The underlying idea is that there is some probability density function from which our datapoints are drawn. Mean Shift then places centroids of clusters at the maxima of that density function in feature space. We get the density functions from kernel density estimation techniques https://en.wikipedia.org/wiki/Kernel_density_estimation. Therefore, the most important parameter of mean shift is the **bandwidth of the kernel**.  

* *robustness to noise and outliers*: ok

* *Occam's razor*: bandwidth of the kernel may be easier to guess than the number of clusters - you can get a feeling for it by looking at the distance matrix of the pairwise distances of points, for example.

* *stability*: usually robust

* *performance with high-dimensional data*: slow

* *performance scalability*: good


In [ ]:
plot_clusters2D(data2D, cluster.MeanShift, (0.175,), {'cluster_all':False})
plot_clustersIRIS(iris, cluster.MeanShift, (2,), {'cluster_all':False})

# Spectral clustering

Spectral clustering is a two-stage process, in which in the first stage a graph will be constructed based on the feature distances between datapoints. Spectral clustering will use some mathematical property of this graph (in particular, its Laplacian) to find a (potentially lower-dimensional) embedding of the graph into Euclidean space. As we can see from this description, this algorithm combines manifold learning with clustering - actually, the main step is the embedding, which in most cases is followed by standard k-means in the transformed space.

* *robustness to noise and outliers*: slightly better than k-means, but this is still at heart a partitional approach if k-means is used as the second stage

* *Occam's razor*: same as k-means

* *stability*: a bit more robust than k-means, since the embedding step can take care of the some initialization differences

* *performance with high-dimensional data*: not so good

* *performance scalability*: ok


In [ ]:
plot_clusters2D(data2D, cluster.SpectralClustering, (), {'n_clusters':6})
plot_clustersIRIS(iris, cluster.SpectralClustering, (), {'n_clusters':3})

# Agglomerative clustering - hierarchical clustering

This is a family of techniques in which clusters are created by iteratively assigning points to clusters based on some kind of pair-wise distance matrix. The distance matrix is updated in each step - this means that we need to have some kind of measure of how to measure  **distances between clusters**. There are many ideas how to do this (take the average of all distances between clusters, or their minimum, or a weighted distance, etc.).

One good element of this technique that it is possible to visualize the clustering using a hierarchy from the sequence of merging points together. 

![title](images/tree_of_life.jpg)

* *robustness to noise and outliers*: not so good, but can potentially be alleviated by stopping early

* *Occam's razor*: if you want clusters, you need to pick when to stop the merging process, so that's the same as k-means

* *stability*: good

* *performance with high-dimensional data*: good

* *performance scalability*: good


In [ ]:
plot_clusters2D(data2D, cluster.AgglomerativeClustering, (), {'n_clusters':6, 'linkage':'ward'})
plot_clustersIRIS(iris, cluster.AgglomerativeClustering, (), {'n_clusters':3, 'linkage':'ward'})

# DBSCAN

DBSCAN tries to find dense clusters - hence it is **not** a partitioning algorithm like many of the ones we looked at above. It first tries to transform the feature space by leaving points in dense regions alone, whereas points in sparse regions are moved further away in its distance metric. We then apply standard agglomerative clustering to the transformed space, which gives us a dendrogram that we then cut at a certain similarity threshold to get the clusters (this parameter is called *epsilon*). 

When we cut the dendrogram, we necessarily get some clusters that only contain a single point. These clusters are discarded as **noise clusters**.  

In the following, if we set *epsilon=100*, all points will be noise-clusters. If we set *epsilon=200*, we get one cluster of points (2,5), and we set *epsilon=220*, we get two clusters (2,5) and (3,4)

![title](images/dendrogram.png)

* *robustness to noise and outliers*: good

* *Occam's razor*: epsilon can be hard to pick, but it is sensitive to the exact choice, the density-based transformation depends on min_samples in the `sklearn` implementation as well

* *stability*: stable across runs, but **not stable** across parameters!

* *performance with high-dimensional data*: good! this algorithm lends itself to various performance tricks because of the local processing that is going on. 

* *performance scalability*: good


In [ ]:
plot_clusters2D(data2D, cluster.DBSCAN, (), {'eps':0.025})
# played with parameters for IRIS
plot_clustersIRIS(iris, cluster.DBSCAN, (), {'eps':0.8})

# HDBSCAN

This is an improved version of DBSCAN. First the algorithm does a density transformation, followed by agglomerative clustering in the transformed space. For treating the dendrogram, it takes a different approach: the dendrogram is condensed by viewing splits that result in a small number of points splitting off as points that will easily leave a cluster and hence may be noise points. From this we get a smaller dendrogram that potentially has fewer clusters that contain such noise points. Overall, the process allows the dendrogram to be cut at **different heights**. 

The parameter of HDBSCAN is now min_cluster_size, which is used in the final step to determine the minimum number of points that are in a cluster. 

* *robustness to noise and outliers*: very good usually, since we actually improved on DBSCAN

* *Occam's razor*: rather intuitive parameter that does, however, depend on the dataset

* *stability*: stable across runs and parameters

* *performance with high-dimensional data*: ok-ish, but not as good as k-means

* *performance scalability*: ok

In [ ]:
import hdbscan
plot_clusters2D(data2D, hdbscan.HDBSCAN, (), {'min_cluster_size':15})
# same parameters for IRIS
plot_clustersIRIS(iris, hdbscan.HDBSCAN, (), {'min_cluster_size':15})